# NBA Sports Betting Model

-----

**Regression model**

> [By: Graham Pinsent](https://twitter.com/GrahamPinsent)  
> [Linkedin](https://www.linkedin.com/in/graham-pinsent/)  
> [GitHub](https://github.com/PerryGraham)

**Tools used:** Python, scrapy,  pandas, sklearn, xgboost, matplotlib, seaborn, numpy 

The purpose of this notebook is to try to predict the [Over/Under](#explination) of an NBA game given the teams previous outcomes. I do not expect to out predict the lines given by the betting websites. However, the main goal of this project is to improve my understanding of regression analysis and get some practice with common practices. If you have any advice or improvements that you would have made please leave a comment and let me know. I'm always happy to get feedback. 

**TLDR:**  
"They are both high scoring teams, this game is going to be high" - The data from this notebook shows that this is not an accurate measurement to predict the total points of an upcoming game. There is a lot of variance when it comes to the total points scored in a match. Even the betting lines are not highly accurate. 

### TABLE OF CONTENTS:
* [Data Collection & Cleaning](#clean)
* [Data Scraping](#scrape)
* [Preprocessing & Notebook start](#pre)
* [Data Insights & Feature Engineering](#feat)
* [Models & Validation](#model)
* [Results](#results)

<sub> I tried changing many features/models/methods throughout this project and left a lot of stuff out of this notebook to make it clean, but I will make notes describing most of the things I tried. Some of the other files with cleaning and trying different features can be found on my github, link above. </sub>

-----

<a name="explination"> </a>
##### What does Over/Under mean? 

The Over/Under is a betting line offered where the betting website will provide a number which you must pick if the outcome will be over or under that given number. This betting model will be referring to the most common over/under value, which is the total points (sum of both teams) scored at the end of the game. For example: if the  over/under value is 225 and you think the outcome will be lower than this, you will take the under. Then after the game team A scores 110 and team B scores 105 (110 + 105 = 115) you have won the bet because 115 is under 225. 

--------

<a name="clean"> </a>
# Data Collection & Cleaning :

* Model data downloaded from Kaggle from [Nathan Lauga](https://www.kaggle.com/nathanlauga/nba-games). 
* Betting line history data scraped from [SDQL](https://sdql.com/) using [scrapy](https://scrapy.org/)

Since the data provided was split up into a few different csv files, I had to do some matching and joining using pandas in order to add a few desired features like team records (wins, loses, gamesplayed). If you would like to see exactly what I did refer to my [github](https://github.com/PerryGraham/Betting-Model-NBA). I will only put some important snippets of the process in this notebook. 

One very important thing I added to the original dataset was the total points scored in the game, which is the target variable of this model. 

```python
gamesdata = pd.read_csv("data/games.csv")

point_total = gamesdata.PTS_home + gamesdata.PTS_away
gamesdata["point_total"] = point_total
```

Then I restricted my data to start in 2011 -> present 

```python
gamesdata = gamesdata.loc[gamesdata.SEASON >= 2011]
gamesdata.sort_values(by=["GAME_DATE_EST"])
gamesdata = gamesdata.reset_index()
```

Some columns had some input errors, for example the season years had an extra number in front (eg: 2016 was in the file as 22016). Simple fix, just subtract 20,000 from all the rows. 

```python
recorddata["SEASON_ID"] = recorddata["SEASON_ID"].apply(lambda x: x - 20000)
```

<a name="here"> </a>
There were some more tricky features that I tried adding, for example I wanted to have a feature that was the point total sum of the last 10 home and 10 away games for each visitor and away team. The problem here is that at the beginning of the season 20 previous games don't exist, so the solution to this was to go back a season. So I had to make a separate dataframe including one year prior to obtain previous years data. After testing this feature I tried changing it to the previous 50 games (25:25, home:away) and this performed much better than only 20. Here is some of the code showing how I did this: 

``` python
avgtotal_home = []

for ind in range(len(gamesdata)):
    date = gamesdata["GAME_DATE_EST"][ind]
    hteam = gamesdata["HOME_TEAM_ID"][ind]
    
    # When they are the home team 
    match_stats = gamesdata.loc[
        (gamesdata.GAME_DATE_EST < date) & (gamesdata.HOME_TEAM_ID == hteam)
    ]
    
    # Home team 25 home games 
    if not match_stats.empty:
        prevgamescount = len(match_stats.index)  # How many games have been played
    if match_stats.empty or prevgamescount < 25:
        # If there are less than 25, then go back to the previous season and take the average
        lastseas = gamesdata2017.loc[
            (gamesdata2017.GAME_DATE_EST < date) & (gamesdata2017.HOME_TEAM_ID == hteam)
        ]
        lastseas10 = lastseas.iloc[:25]
        avglast5home = lastseas.point_total.mean(axis=0)
    else:
        only5 = match_stats.iloc[:25]
        avglast5home = only5.point_total.mean(axis=0)

    avgtotal_home.append(avglast5home)
    # This list will be added as a column to the dataframe 
```

Once I had the average of the previous 50 games for both teams in the matchup. I then took the mean of those two numbers. The idea here is that you should be able to see based on past performance if the team often has high scoring games or not. Then if you take the mean of both teams you should get a good value for predicting if the game will be high or low scoring. This value, along with the season, ended up being the strongest features in my model:

```python
mean = ((gamesdata.avgpointtotal_away + gamesdata.avgpointtotal_home) / 2)
gamesdata["meanpointtotal"] = mean 
```

-----------

# Data Scraping <a name="scrape"> </a>

I would like a baseline value to be able to compare to the accuracy of my model. So I decided to scrape previous betting line data and compare it to the outcome in order to calculate the mean absolute error of the betting line. I use the [scrapy](https://scrapy.org/) library to do this. Here is the code for the web scraper: 

```python
class tableSpider(scrapy.Spider):
    name= 'table'
    
    start_url =[
        "https://sportsdatabase.com/nba/query?output=default&sdql=date%2C+team%2C+site%2C+o%3Ateam%2C+total%2C++points%2C+o%3Apoints+%40season%3E2010&submit=++S+D+Q+L+%21++"
    ]

    def parse(self, response):
        for row in response.xpath('//*[@class="dataTables_wrapper no-footer"]//tbody/tr'):
            if row.xpath('td[3]//text()').extract_first() == "home":
                yield {
                    'date' : row.xpath('td[1]//text()').extract_first(),
                    'team' : row.xpath('td[2]//text()').extract_first(),
                    'site' : row.xpath('td[3]//text()').extract_first(),
                    'o:team' : row.xpath('td[4]//text()').extract_first(),
                    'total' : row.xpath('td[5]//text()').extract_first(),
                    'points' : row.xpath('td[6]//text()').extract_first(),
                    'o:points' : row.xpath('td[7]//text()').extract_first(),
                } 
```

From here getting the mean absolute error of the betting lines was easy: 

In [ ]:
import pandas as pd 

data = pd.read_csv("../input/betscsv/bets.csv")

# The rows have duplicates because of home/away, so remove every other row
data = data[data.site == "home"]

# Adding points for both teams together
data["real_total"] = data["points"] + data["o:points"]

# Calculating the difference between the line and outcome 
data["error"] = data["real_total"] - data["total"]

# Making the error the absolute value
data["error"] = data.error.abs()

# Calculating mean absolute error between the lines and outcome 
n = len(data)
total_error = data.error.sum()
mae_lines = total_error / n

print ("MAE over last 10 years",mae_lines)

-------

# Preprocessing & EDA <a name="pre"> </a>

I decided to one-hot encode the team ids, this seems like the best way to handle the home and away teams. Although it adds a lot of columns to the dataframe. For the Season, I just scaled it down to start at 0. Then dropped the old columns, and the points scored per team because this is unknown untill the game is over, which would cause data leaks. 

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from sklearn import preprocessing 

# Read the data
X_full = pd.read_csv("../input/cleandata2csv/cleandata2.csv")
X_full["GAME_DATE_EST"] = pd.to_datetime(X_full["GAME_DATE_EST"],infer_datetime_format=True)
X_full = X_full.sort_values(by=["GAME_DATE_EST"], ascending=False)
X_full = X_full.reset_index()



# Label / One-Hot encoding team IDs and season
le = preprocessing.OneHotEncoder()
le2 = preprocessing.LabelEncoder()
ohe_home = le.fit_transform(X_full[["HOME_TEAM_ID"]]).toarray()
ohe_away = le.transform(X_full[["VISITOR_TEAM_ID"]]).toarray()
X_full["SEASON"] = X_full["SEASON"] - 2010

# Drop old columns
X_full = X_full.drop(columns =[
    "HOME_TEAM_ID","VISITOR_TEAM_ID", "PTS_home", "PTS_away","GAME_DATE_EST","index"], axis=1)

Here I tried a few different operations to the average point total columns to see if I could get a useful feature from these. Turns out that the difference between the two happens to be a strong feature that helped get a bit better performance in my models. 

In [ ]:
# Testing features:
X_full["diff"] = X_full["avgpointtotal_home"] - X_full["avgpointtotal_away"]
#X_full["diff"] = X_full["diff"]**2
#X_full["diff"] = X_full["diff"].abs()
#X_full["multi"] = X_full.apply(lambda row: row.avgpointtotal_home / row.avgpointtotal_away, axis=1)

#X_full["off_power_home"] = X_full["point_average_last10"] - X_full["away_point_againts_average_last10"]
#X_full["off_power_away"] = X_full["away_point_average_last10"] - X_full["point_againts_average_last10"]
#X_full["gap"] = X_full["off_power_home"].abs() + X_full["off_power_away"].abs()

In [ ]:
X_full.head(5)

Here we can see the layout of our data so far. I will explain what some of the columns mean:  
**point_average_last10** - is the average points scored for the home team in the last 10 games   
**points_againts_average_last10** - is the average points that got scored against the home team in the past 10 games  
The next two columns are the same but for the visitor team.   
These columns should act as a way to identify strong defensive teams / strong offensive teams (which should hopefully indicate the outcome of the current game)  
**cpg** - current games played (how many games they have completed this season) (_away is the same but for the visitor team)  
last 4 columns are the ones described earlier ([here](#here))


In [ ]:
X_full.describe()

In [ ]:
X_full.point_total.describe()

One thing I notice here is that there is a lot of variance in the target variable. The standard deviation is ~22 and the range from Q1-Q3 is 30.

Checking to see the correlation between all of the features. As expected there is a lot of correlation between the features. It is important to keep this in mind when selecting models/parameters.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(16, 12))
sns.heatmap(X_full.corr(),
            cmap="Blues",annot=True, fmt='.2f', vmin=0);

There are some highly correlated features, these will be addressed later. The features with the highest correlation with the target are: season, meantotalpoints, and point_average_last10/away_points_average_last10

Adding on the one-hot encoded columns: 

In [ ]:
ohe_home_df = pd.DataFrame(ohe_home)
ohe_away_df = pd.DataFrame(ohe_away, columns=list("abcdefghijklmnopqrstuvwxyzABCD"))
X_full = pd.concat([X_full,ohe_home_df], axis=1)
X_full = pd.concat([X_full,ohe_away_df], axis=1)
print(X_full.shape)

In [ ]:
X_full.head(5)

-----------

# Feature Engineering <a name="feat"> </a>

After trying a lot of different features, adjusting how many games to go back when taking the mean; here are the most successful ones that I could get. I tried removing many of the features and trying different combinations. The best results are when I used more data and features. 

Let's take a look at some relationships between features and target:

In [ ]:
import matplotlib.pyplot as plt

with plt.style.context("ggplot"):
    plt.scatter(X_full.meanpointtotal, X_full.point_total, marker="o", alpha=0.1, color='#9467bd')
    plt.xlabel("Mean last 50 games")
    plt.ylabel("Total points (y)")
    plt.title("Total points (y) vs mean last 50 games ")
fig1 = plt.figure();

There is a lot of noise and this isn't the strongest correlation. However, there is a clear trend between the total points scored and the average of both teams' last 50 games played. 

In [ ]:
with plt.style.context("ggplot"):
    plt.scatter(X_full["diff"], X_full.point_total, marker="o", alpha=0.1)
    plt.xlabel("Difference")
    plt.ylabel("Total points (y)")
    plt.title("Total points (y) vs mean last 50 games ")
fig2 = plt.figure();

This graph shows that there is a slight trend towards a medium scoring game when there is a large difference between the two teams. But if the teams are closer in terms of their average last 50 games, then it could be a high or low scoring game. 

This trend is a bit easier to see when you plot the absolute value of the difference. 

In [ ]:
with plt.style.context("ggplot"):
    plt.scatter(X_full["diff"].abs(), X_full.point_total, marker="o", alpha=0.1)
    plt.xlabel("Difference")
    plt.ylabel("Total points (y)")
    plt.title("Total points (y) vs mean last 50 games ")
fig3 = plt.figure();

In [ ]:
#mean_df = X_full.copy()
#mean_df["target"] = y
season_avg = X_full.groupby(by=["SEASON"]).mean()

with plt.style.context("ggplot"):
    plt.scatter(season_avg.index, season_avg.point_total, color = "#d62728" )
    plt.xlabel("Season")
    plt.ylabel("Total points (y)")
    plt.title("Mean Total Points Per Season")
fig4 = plt.figure();

On average the amount of points scored are going up over time. Season 1 refers to 2011, all the way to 2019 (9)

--------------

# Models <a name="model"> </a>

The method for validation that I chose was doing a manual 90:10 train test split with no random shuffling. That is, when we are using all of the old data to train and the newest 10% to validate. This was chosen because it was the simplest way to achieve a good validation test for this type of model. Cross validation is not a good idea because then you would be training on data that you would not have access to if you were to use this model in practice. For example, you might be able to train on 2019 data and see that that is a high scoring season, then when predicting an early 2019 game you would get better results than you should. In theory the best method would be a leave future out validation method, because that is how the model would operate in practice. For simplicity I used 90:10. Which should yield decent enough results for the task. 

In [ ]:
y = X_full["point_total"]
X_full = X_full.drop(columns=['point_total'])

test = len(X_full) - int(len(X_full)*0.9)

X_train = X_full.iloc[test:]
X_valid = X_full.iloc[:test]
y_train = y.iloc[test:]
y_valid = y.iloc[:test]



In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error

EN = ElasticNet().fit(X_train,y_train)

pred_EN = EN.predict(X_valid)
mae_EN = mean_absolute_error(pred_EN, y_valid)
print ("mae",mae_EN)

In [ ]:
from sklearn.linear_model import Lasso 

ls = Lasso().fit(X_train, y_train)

pred_ls = ls.predict(X_valid)
mae_ls = mean_absolute_error(pred_ls, y_valid)
print("mae",mae_ls)

In [ ]:
from sklearn.linear_model import Ridge

rg = Ridge().fit(X_train, y_train)

pred_rg = rg.predict(X_valid)
mae_rg = mean_absolute_error(pred_rg, y_valid)
print("mae",mae_rg)

In [ ]:
from sklearn.linear_model import TheilSenRegressor

ts = TheilSenRegressor().fit(X_train, y_train)
pred_ts = ts.predict(X_valid)
mae_ts = mean_absolute_error(pred_ts,y_valid)
print("mae",mae_ts)

In [ ]:
#lin reg 
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import cross_val_score

lin = LinearRegression(normalize=True, ).fit(
    X_train,y_train
)
pred_lin = lin.predict(X_valid)
mae_lin = mean_absolute_error(pred_lin, y_valid)
print("mae", mae_lin)

In [ ]:
from sklearn.neural_network import MLPRegressor

regr = MLPRegressor(random_state=1, max_iter=1000).fit(
    X_train, y_train
)

pred_mlp = regr.predict(X_valid)
mae_regr = mean_absolute_error(pred_mlp, y_valid)
print("mae",mae_regr)

In [ ]:
import xgboost as xgb

xg = xgb.XGBRegressor(
    booster="gblinear",
    objective="reg:squarederror",
    base_score="1",
    eval_metric="cox-nloglik"
    ).fit(X_train,y_train)
pred_xg = xg.predict(X_valid)
xg_regr = mean_absolute_error(pred_xg, y_valid)
                         
print("mae",xg_regr)
print("done")

Calculating the MAE for guessing the mean of last season every time. 

In [ ]:
error = []
mean_last_season = season_avg.iloc[6,0]
for i in y_valid:
    error.append(abs(mean_last_season - i))
mae_mean = sum(error) / len(error)

print(mae_mean)
print(mean_last_season)

Using [hpsklearn](https://github.com/hyperopt/hyperopt-sklearn/tree/master/hpsklearn) library to try a collection of models with **hyperoptimization** of parameters: 

In [ ]:
from hpsklearn import HyperoptEstimator, any_regressor
from hyperopt import tpe

estim = HyperoptEstimator(regressor=any_regressor("svr"))

estim.fit(X_train.values, y_train.values)

print(estim.best_model())

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_testing = scaler.fit_transform(X_train)
X_testing_valid = scaler.transform(X_valid)

In [ ]:
xg1 = xgb.XGBRegressor(base_score=0.5, booster='gbtree',
             colsample_bylevel=0.8886553695663921, colsample_bynode=1,
             colsample_bytree=0.6068338278675369, gamma=0.0017889282555567038,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.0013963222902296055, max_delta_step=0, max_depth=7,
             min_child_weight=19, monotone_constraints='()',
             n_estimators=5400, n_jobs=0, num_parallel_tree=1,
             objective='reg:linear', random_state=1,
             reg_alpha=0.22448614695919908, reg_lambda=2.9482948529431052,
             scale_pos_weight=1, seed=1, subsample=0.8571414026048771,
             tree_method='exact', validate_parameters=1, verbosity=None).fit(X_testing,y_train)
        
pred_xg1 = xg1.predict(X_testing_valid)
xg1_regr = mean_absolute_error(pred_xg1, y_valid)
print("mae",xg1_regr)

-----------
<a name="results"> </a>

# Results 

In [ ]:
results = {"Model": ["ElasticNet","LinearRegression",
                     "MLPRegression","XGBoost",
                     "Lasso","Ridge","TheilSen",
                     "Betting lines","Guessing the mean",
                     "HyperoptEstimator"],
          "MAE Score": [mae_EN,mae_lin,
                       mae_regr,xg_regr,mae_ls,
                       mae_rg,mae_ts,mae_lines,
                       mae_mean,xg1_regr]}
result_df = pd.DataFrame(data=results)
result_df = result_df.sort_values(by=["MAE Score"])
print(result_df.to_string(index=False))

#### The best performer is the **Hyperoptimized XGBoostRegressor**

Interpretation of the models accuracy: 

In [ ]:
bttm = (1 - (xg1_regr / mae_mean))*100
wttl = (1 - (mae_lines / xg1_regr))*100
print("The model is {:.0f}% better than guessing the mean".format(bttm))
print("The model is {:.0f}% worse than the betting lines".format(wttl))

There is so much noise in the target variable that predicting the total points scored in an NBA game is not easy. Using past performances as an estimator for point total does not work very well. "They scored 130 points last game, it's going to be high again" - this data shows that this is clearly a flawed statement. Even the best models from the betting site have a mean absolute error of 13.40  points. 

##### Visualization of model predictions:

In [ ]:
import numpy as np
with plt.style.context("ggplot"):
    plt.scatter(range(len(pred_xg1)), pred_xg1, marker="o", alpha=0.3,color = "#0066ff")
    plt.xlabel("Games")
    plt.ylabel("Estimated total points (y)")
    plt.title("Best Model Predictions")
fig6 = plt.figure();

An explanation of why the predictions look like this is because the first ~400 games are from the 2018 season and the rest is the 2019 season. 

##### Visualization of actual outcomes:

In [ ]:
with plt.style.context("ggplot"):
    plt.scatter(range(len(y_valid)), y_valid, marker="o", alpha=0.3, color="#0066ff")
    plt.xlabel("Games")
    plt.ylabel("Total points (y)")
    plt.title("Real Outcomes")
fig7 = plt.figure();


They look similar but the x-axis scale is much larger for the real outcomes graph. 

To see if our best model is predicting higher or lower than real outcome we can calculate the mean error without taking the absolute value: 

In [ ]:
ls = []
for i in range(len(pred_ts)):
    ls.append(pred_xg1[i] - y_valid.iloc[i])
print ("Average Error of model:",sum(ls))
with plt.style.context("ggplot"):
    plt.scatter(range(len(ls)), ls, marker="o", alpha=0.3, color="#ff0000")
    plt.xlabel("Games")
    plt.ylabel("Prediction-Outcome Difference")
    plt.title("Error")
fig7 = plt.figure();

A positive value means that the model on average is predicting higher than the real outcome. This can sometimes be insightful to see what the model is doing and adjust accordingly. Notice that the negative error values spread wider than the positive, this is because of the overtime games. 

**Things I would have done differently if I were to do this again**:

* Just pick one team, try to predict the total points of their games. Instead of doing every team and every match. Perhaps this would allow me to explore more features and focus on the specific trends of that team and matchups. 

* Identify games that went to overtime, and have the total points when the regular time ended. This could help reduce some overtime results  bias. 


People that helped: 

[Benjamin Retser](https://www.kaggle.com/benjaminretser)


Thank you 